<a href="https://colab.research.google.com/github/sadra-barikbin/persian-information-retrieval-example/blob/main/Persian-IR-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install hazm transformers ir_measures
!pip install -q clean-text[gpl]

In [61]:
import torch
import yaml
import hazm
import numpy as np
import pandas as pd
import ir_measures as IRm
from typing import List, Tuple
from pathlib import Path
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Loading & Preparing Data

## Corpus

In [ ]:
!wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip

In [ ]:
!unzip doc_collection.zip

In [64]:
!cat IR_dataset/1000.txt

ببر سیبری که با نام‌های ببر آلتایی، ببر منچوری، ببر کره‌ای، ببر آمور و ببر اوسوری نیز شناخته می‌شود، یکی از زیرگونه‌های ببر است که در گذشته در بخش‌های وسیعی از شرق آسیا می‌زیست اما امروزه تنها در منطقهٔ حفاظت شده‌ای در شرق سیبری زندگی می‌کند. ببر سیبری بزرگترین زیرگونهٔ ببر و بزرگترین گربه‌سان زندهٔ جهان است. ببر منقرض شده مازندران نزدیک‌ترین زیرگونه ببر به ببر سیبری است و مطالعات ژنتیکی جدید حکایت از آن دارد که این دو را حتی می‌توان یک زیرگونه محسوب کرد.

ببر سیبری در دهه ۱۹۳۰ در آستانه انقراض قرار داشت و تعداد آن‌ها تنها به بیست تا سی ببر کاهش یافته بود. اما این حیوان به طرزی باورنکردنی از انقراض قریب‌الوقوع رهایی جست و جمعیت آن تا سال ۲۰۱۰ به حدود ۳۶۰ ببر رسید. ببر سیبری با توجه به همین افزایش جمعیت از سال ۲۰۱۰ از بالاترین ردهٔ حفاظتی یعنی «به شدت در معرض خطر» خارج شده و در یک رده پایین‌تر یعنی «در خطر انقراض» قرار گرفته است. ببرهای سیبری تنوع ژنتیکی بسیار پائینی دارند که این به دلیل کاهش شدید جمعیت این حیوان در دهه ۱۹۴۰ و تعداد اندک توله ببرهایی است که به بلوغ می‌رسند. ضمن اینکه بی

In [65]:
corpus = [(int(path.stem), path.open().read()) for path in Path('IR_dataset').iterdir()]
corpus = pd.DataFrame(corpus, columns=['docId','text']).set_index('docId').sort_index()

In [66]:
ccorpus = [(int(path.stem), path.open().read()) for path in Path('IR_dataset').iterdir()]

In [67]:
corpus.head()

,text
docId,
0,برخی از هواداران مصدق یا اعضای جبهه ملی که در ...
1,جبهه ملی ایران که به اختصار جبهه ملی نیز خواند...
2,سرلشکر زاهدی در سال ۱۳۲۸ و پس از آن‌که دخالت‌ه...
3,نمایندگان طرفدار مصدق در حمایت از ابقای دولت و...
4,نمایندگان طرفدار مصدق در حمایت از ابقای دولت و...


## Qrels

In [68]:
!wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml

--2021-12-30 18:10:12--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50854 (50K) [text/plain]
Saving to: ‘evaluation_IR.yml.1’

evaluation_IR.yml.1 100%[===================>]  49.66K  --.-KB/s    in 0.01s   

2021-12-30 18:10:12 (4.92 MB/s) - ‘evaluation_IR.yml.1’ saved [50854/50854]



In [69]:
query_raw_data = yaml.safe_load(open('evaluation_IR.yml'))

In [70]:
query = pd.Series(query_raw_data.keys())
qrels = [{'query_id':idx, 'doc_id':d,
          'relevance':3} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_high']]
qrels.extend([{'query_id':idx, 'doc_id':d,
          'relevance':2} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_med']])
qrels.extend([{'query_id':idx, 'doc_id':d,
          'relevance':1} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_low']])
qrels.extend([{'query_id':idx, 'doc_id':query_raw_data[q]['relevant'][0],
          'relevance':4} for idx,q in query.to_dict().items()])
qrels = pd.DataFrame(qrels)

In [71]:
query[147],query_raw_data[query[147]]

('گرجستان  تاریخ',
 {'relevant': [388],
  'similar_high': [389, 390, 391, 392, 393, 394],
  'similar_low': [404, 405, 406, 407, 408, 409, 410, 411, 412, 413],
  'similar_med': [395, 364, 396, 397, 398, 399, 400, 401, 402, 403]})

In [72]:
qrels.sample(n=5).reset_index(drop=True)

,query_id,doc_id,relevance
0,126,463,2
1,111,908,2
2,137,2133,1
3,131,2584,1
4,74,1318,1


## Normaliztion

In [73]:
normalize = hazm.Normalizer().normalize
corpus.text = corpus.text.transform(normalize)
query = query.transform(normalize)

# Embedding the documents

## Method 1 : Tfidf

In [74]:
vectorizer = TfidfVectorizer(max_features=500,ngram_range=(1,2))
vectorizer.fit(corpus.text)

TfidfVectorizer(max_features=500, ngram_range=(1, 2))

## Method 2 : ParsBert

In [102]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

model = AutoModel.from_pretrained(model_name_or_path)
model = model.cuda()

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

In [103]:
text = "ما در قرن ۲۱ زندگی می‌کنیم" 
encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      max_length = 500,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',  # Return PyTorch tensors
    )
out = model(
            input_ids = encoding['input_ids'].cuda(), 
            attention_mask= encoding['attention_mask'].cuda())
out['pooler_output'][0]

tensor([-6.1199e-01, -7.2380e-01,  3.3152e-01, -8.0473e-01,  1.4409e-02,
         3.2757e-01,  2.5225e-02,  4.0572e-01, -2.3879e-01,  3.6961e-01,
         1.1989e-01, -8.7340e-01, -7.1609e-02, -3.3276e-01,  1.4152e-01,
        -1.0844e-01,  8.6605e-03, -8.1835e-03,  4.7999e-01,  6.7156e-01,
        -5.7034e-01,  3.6085e-03,  3.8205e-02,  2.4517e-01,  4.1049e-01,
         4.9698e-01, -4.9224e-01, -4.4647e-02, -6.6256e-02,  3.0444e-01,
        -2.3789e-01, -2.4569e-01,  2.6970e-01,  7.7225e-01, -5.1137e-01,
         7.7780e-01,  1.2946e-02,  5.1527e-01, -1.6624e-01,  5.2252e-01,
         7.1204e-01, -6.8531e-01,  3.8264e-01, -9.7279e-02,  1.8490e-02,
         5.6935e-01, -8.2118e-01,  2.3006e-01,  1.1544e-01, -2.7786e-01,
        -4.8266e-01,  3.2057e-01, -4.5657e-01,  3.3598e-01, -4.9861e-01,
        -1.9419e-01,  4.5624e-01,  1.9863e-01, -7.4756e-01,  3.5362e-01,
        -8.1764e-02, -2.6380e-01, -4.6832e-01, -4.3649e-01,  1.3785e-01,
         7.0854e-01, -3.3658e-01,  8.6266e-02,  1.7

In [82]:
def get_embed(part):
  encoding = tokenizer.encode_plus(
    part,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    max_length = 500,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',  # Return PyTorch tensors
  )
  out = model(
      input_ids = encoding['input_ids'].cuda(), 
      attention_mask= encoding['attention_mask'].cuda())
  return out['pooler_output'].cpu().detach().numpy()

In [29]:
doc_vec = np.zeros((1, 768))
doc_map = np.zeros(1)
import tqdm

for index, doc in tqdm.tqdm(corpus.iterrows()):
  doc_split = doc['text'].split()
  doc_parts = [' '.join(doc_split[i:i + 300]) for i in range(0, len(doc_split) - 150, 150)]
  for part in doc_parts:
    doc_vec = np.append(doc_vec, get_embed(part), axis = 0)
    doc_map = np.append(doc_map, doc['docId'])


3258it [13:06,  4.14it/s]


# Document Retrieval

In [83]:
class KNN_based_IR(BaseEstimator):
  def __init__(self,n_neighbors=1+10+10+10) -> None:
    super().__init__()
    self.nn = NearestNeighbors(n_neighbors=n_neighbors)
  def set_params(self,**kwargs):
    self.nn.set_params(**kwargs)
  def fit(self, X: np.array):
    self.nn.fit(X)
  def predict(self, X: np.array):
    distances, docIds = self.nn.kneighbors(X)
    scores = np.max(distances)-distances
    return scores, docIds

In [84]:
IR_system = KNN_based_IR()
IR_system.fit(vectorizer.transform(corpus.text))

In [100]:
bert_knn = KNN_based_IR(50)
bert_knn.fit(doc_vec)

# IR Evaluation
Tailored for our multi-level Test Collection.

In [87]:
preds = IR_system.predict(vectorizer.transform(query))

In [101]:
bert_pred = []
for q, ret in tqdm.tqdm(query_raw_data.items()):
  pr = get_embed(q)
  res = [doc_map[i] for i in bert_knn.predict(pr)[1][0] if i != 0]
  bert_pred.append(res)

100%|██████████| 150/150 [00:06<00:00, 23.24it/s]


## Adapting IR output to our Test Collection

In [89]:
def adapt_IR_output_to_measure_input(IR_output: Tuple[np.array, np.array]):
  scores, docIds = IR_output
  return pd.DataFrame({'query_id': np.tile(query.index,(31,1)).flatten(order='F').astype(str),
                       'doc_id':   docIds.flatten().astype(str),
                       'score':    scores.flatten()})

## MRR (Mean Reciprocal Rank)

In [90]:
MRR = IRm.measures.MRR()
def mrr_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return MRR.calc_aggregate(qrels[qrels.relevance == 4], ret)
# mrr_scorer = make_scorer(mrr)

In [91]:
mrr_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.11047063325391496

## MAP (Mean Average Precision)

In [96]:
def map_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return np.mean([IRm.measures.AP(rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret) for level in range(1,4+1)])

# map_scorer = make_scorer(map)

In [97]:
map_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.06630537139767945

## P@K

In [98]:
def p_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return np.mean([IRm.measures.P(cutoff=k, rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret)\
                  for k,level in zip([1,11,21,31],range(1,4+1))])

In [99]:
p_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.033872596937113045

# Pipeline Definition

In [ ]:
pipeline = Pipeline([('embedding','passthrough'),
                     ('retrieval','passthrough')])